<img src="../images/logos/arm_logo.png" width=500 alt="ARM Logo"></img>

# Snowfall Retrievals from SAIL X-Band Radar

In [2]:
import warnings
warnings.simplefilter("ignore", UserWarning)

import glob
import os
import datetime

import numpy as np
import matplotlib.pyplot as plt
import pyart
import xarray as xr
from matplotlib.dates import DateFormatter

from metpy.plots import USCOUNTIES

import cartopy.crs as ccrs
import cartopy.feature as cfeature


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



## Radar Reflectivity - Snowfall (Z-S) Relationships
We setup helper functions to calculate the snowfall retrieval, using following notation:

$Z = A*S ^ {B}$

Where:
- Z = Reflectivity in dBZ
- A = Coefficient applied to Z-S Relationship (not in the exponent)
- S = Liquid snowfall rate
- B = Coefficient applied to Z-S Relationship (in the exponent)

We also need to apply a snow water equivalent ratio (`swe`) to convert from liquid to snow (ex. 8 inches of snow --> 1 inch of rain would be 8.0).

This equation now becomes:

$Z = swe*A*S ^ {B}$

Solving for S, we get:

$S = swe * (\frac{z}{a})^{1/B}$

Where z is reflectivity in units of dB ($z =10^{Z/10}$)

## Define our Z-S Relationships

We start by using the Z-S Relationships Described in [Bukovčić et al. (2018)](https://doi.org/10.1175/JAMC-D-17-0090.1), where we refer to following relationships in Table 1:

| Source                      | Z(S) relation for dry snow  | A Coefficient | B Coefficient | Radar Band |
| :---                        |           :----:            |          ---: | :---------:   | :----:     |
| Wolfe and Snider (2012)     | $Z = {110}S^{2}$            | 110           |  2            | S          |
| WSR-88D high plains         | $Z = {130}S^{2}$            | 130           |  2            | S          |
| WSR-88D Intermountain West  | $Z = {40}S^{2}$             | 40            |  2            | S          |
| Matrosov et al.(2009) Braham(1990)   | $Z = {67}S^{1.28}$ | 67            |  1.28         | X          |
| Matrosov et al.(2009) Braham(1990)   | $Z = {114}S^{1.39}$| 114           |  1.39         | X          |
| Matrosov et al.(2009) Braham(1990)   | $Z = {136}S^{1.3}$ | 136           |  1.3          | X          |
| Matrosov et al.(2009) Braham(1990)   | $Z = {28}S^{1.44}$ | 28            |  1.44         | X          |
| Matrosov et al.(2009) Braham(1990)   | $Z = {36}S^{1.56}$ | 36            |  1.56         | X          |
| Matrosov et al.(2009) Braham(1990)   | $Z = {48}S^{1.45}$ | 48            |  1.45         | X          |
| Matrosov (1992)             | $Z = {410}S^{1.6}$          | 410           |  1.6          | X          |
| Matrosov (1992)             | $Z = {340}S^{1.6}$          | 340           |  1.75         | X          |
| Matrosov (1992)             | $Z = {240}S^{1.6}$          | 240           |  1.95         | X          |
| Boucher and Wieler (1985)   | $Z = {229}S^{1.65}$         | 229           |  1.65         | X          |
| Fujiyoshi et al. (1990)     | $Z = {427}S^{1.09}$         | 427           |  1.09         | X          |
| Puhakka (1975)              | $Z = {1050}S^{2}$           | 1050          |  2            | X          |
| Sekhon and Srivastava (1970)| $Z = {1780}S^{2.21}$        | 1780          |  2.21         | X          |


## List the Available Files
We will use files on the Oak Ridge Laboratory Computing Facility (ORLCF), within the shared SAIL directory `/gpfs/wolf/atm124/proj-shared/sail`.

These radar files have been merged from a single sweep in each file, to whole volume scans in each file.

In [3]:
file_list = sorted(glob.glob("/data/project/ARM_Summer_School_2024_Data/sail/radar/*"))

In [5]:
file_list[0:10]

['/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.000206.nc',
 '/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.000726.nc',
 '/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.001246.nc',
 '/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.001806.nc',
 '/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.002326.nc',
 '/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.002846.nc',
 '/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.003406.nc',
 '/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.003926.nc',
 '/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.004446.nc',
 '/data/project/ARM_Summer_School_2024_Data/sail/radar/gucxprecipradarcmacS2.c1.20220310.00

## Your Turn!

<div class="admonition alert alert-success">
    <p class="admonition-title" style="font-weight:bold">Question 1 - Easy</p>
    Using material presented this week, apply one of the above relationships to SAIL X-Band radar file and display
</div>

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Question 2 - Intermediate</p>
    Using the above methodology, calculate the snowfall accumulation for an hour (or day) 
</div>

<div class="admonition alert alert-danger">
    <p class="admonition-title" style="font-weight:bold">Question 3 - Hard</p>
    Compare this hourly (or daily) accumulation to WRF output for the same day